<a href="https://www.kaggle.com/code/brunokobi/disaster-tweets?scriptVersionId=134968903" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# Carregamento dos Dados

In [2]:
# Leitura dos Dados 
train = pd.read_csv('/kaggle/input/nlpgettingstarted/train.csv')
test = pd.read_csv('/kaggle/input/nlpgettingstarted/test.csv')
submission = pd.read_csv('/kaggle/input/nlpgettingstarted/sample_submission.csv')

In [3]:
# Essas linhas de código permitem extrair os dados de texto dos tweets 
train_data = train.text.values
train_labels = train.target.values
test_data = test.text.values

In [4]:
#Carregamento do modelo Universal Sentence Encoder(USE) da google
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
embed = hub.KerasLayer(module_url, trainable=True, name='USE_embedding')

# Treinamento do Modelo usando a rede neural 

In [5]:
def build_model(embed):
    
    # Criação do modelo sequencial
    model = Sequential([
        Input(shape=[], dtype=tf.string),
        embed,  # Camada de incorporação de texto (por exemplo, USE)
        Dense(1, activation='sigmoid')  # Camada densa de saída com ativação sigmoidal
    ])
    
    # Compilação do modelo
    model.compile(Adam(2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
model = build_model(embed)  # Criação do modelo usando a camada de incorporação de texto fornecida
model.summary()  # Exibir um resumo do modelo

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_embedding (KerasLayer)  (None, 512)               147354880 
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 147,355,393
Trainable params: 147,355,393
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

train_history = model.fit(
    train_data, train_labels,
    validation_split=0.10,
    epochs=2,
    callbacks=[checkpoint],
    batch_size=512
)

Epoch 1/2
14/14 [==============================] - 537s 13s/step - loss: 0.6896 - accuracy: 0.5442 - val_loss: 0.6817 - val_accuracy: 0.6247
Epoch 2/2
14/14 [==============================] - 178s 13s/step - loss: 0.6693 - accuracy: 0.7031 - val_loss: 0.6601 - val_accuracy: 0.7323


In [8]:
model.load_weights('model.h5')  # Carrega os pesos salvos do modelo a partir do arquivo 'model.h5'
test_pred = model.predict(test_data)  # Faz previsões utilizando o modelo carregado nos dados de teste

102/102 [==============================] - 35s 277ms/step


In [9]:
submission['target'] = test_pred.round().astype(int)  # Atribui as previsões arredondadas como inteiros à coluna 'target' do DataFrame 'submission'
submission.to_csv('/kaggle/working/submission.csv', index=False)  # Salva o DataFrame 'submission' como um arquivo CSV chamado 'submission.csv' no diretório '/kaggle/working'
